# 梯度提升树（GBDT）详解

**梯度提升树（Gradient Boosting Decision Tree, GBDT）**是一种强大的集成学习方法，通过**将多个决策树顺序叠加**来不断减小预测误差。

它在机器学习竞赛、工业应用中都有极其广泛的使用。

---

## 一、核心思想

- GBDT是**Boosting + 决策树**。
- 每一轮训练一个新树，拟合上轮模型的残差（负梯度）。
- 所有树的预测结果累加形成最终输出。

---

## 二、算法流程

假设目标是最小化一个损失函数：
$$
L = \sum_{i=1}^n \ell(y_i, F(x_i))
$$

**步骤：**

1. 初始化模型：
   $$
   F_0(x) = \arg\min_\gamma \sum_i \ell(y_i, \gamma)
   $$
   - 回归：通常是样本均值。

2. 对每一轮 $m=1,2,...,M$：
   - 计算负梯度（残差）：
     $$
     r_i^{(m)} = -\left[\frac{\partial \ell(y_i, F(x_i))}{\partial F(x_i)}\right]_{F(x)=F_{m-1}(x)}
     $$
   - 在$(x_i, r_i^{(m)})$上训练一棵回归树$h_m(x)$。
   - 计算最佳步长：
     $$
     \gamma_m = \arg\min_\gamma \sum_i \ell(y_i, F_{m-1}(x_i) + \gamma \cdot h_m(x_i))
     $$
   - 更新模型：
     $$
     F_m(x) = F_{m-1}(x) + \nu \cdot \gamma_m \cdot h_m(x)
     $$
     其中$\nu$是学习率（常取0.01~0.1）。

3. 最终模型：
   $$
   F_M(x) = F_0(x) + \sum_{m=1}^M \nu \cdot \gamma_m \cdot h_m(x)
   $$

---

## 三、支持的损失函数

- 回归：平方误差、绝对误差
- 二分类：对数损失（Logistic Loss）
- 多分类：Softmax

---

## 四、GBDT与随机森林对比

| 特性             | 随机森林                  | GBDT                     |
|------------------|---------------------------|--------------------------|
| 训练方式         | 并行独立训练多棵树        | 串行逐步拟合残差        |
| 偏差/方差        | 降低方差                  | 同时降低偏差和方差      |
| 样本重采样       | Bootstrap采样             | 可选（subsample）        |
| 性能             | 稳定，偏差高              | 更强，易过拟合           |
| 可解释性         | 中等                      | 较差                     |

---

## 五、优缺点

✅ **优点**
- 强大的预测性能
- 支持多种损失函数
- 可评估特征重要性
- 调参灵活

❌ **缺点**
- 串行训练，速度慢
- 对噪声敏感
- 超参数多，需小心调参

---

## 六、Python示例（scikit-learn GBDT）

```python
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# 生成数据
X, y = make_regression(n_samples=1000, n_features=10, noise=0.2, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

# 定义GBDT模型
gbdt = GradientBoostingRegressor(
    n_estimators=100,
    learning_rate=0.1,
    max_depth=3,
    subsample=0.8,
    random_state=42
)

gbdt.fit(X_train, y_train)
y_pred = gbdt.predict(X_test)

print("MSE:", mean_squared_error(y_test, y_pred))
```

---

## 七、调参要点

1. **n_estimators**
   - 弱学习器数量
   - 越大模型越复杂
2. **learning_rate**
   - 学习率（步长）
   - 越小越稳健，需配合增大n_estimators
3. **max_depth**
   - 单棵树深度
   - 控制复杂度
4. **subsample**
   - 子样本比例
   - <1.0时相当于随机抽样，可防过拟合
5. **min_samples_leaf**
   - 每个叶节点最少样本数

> 一般先调`n_estimators`和`learning_rate`，再调`max_depth`。

---

## 八、进阶：XGBoost / LightGBM / CatBoost

GBDT的高性能实现：
- **XGBoost**
  - 支持正则化
  - 并行高效
- **LightGBM**
  - 直方图优化
  - 更快
- **CatBoost**
  - 对类别特征支持好

---

## 九、总结

**GBDT**是一种强大的提升树方法：
- 逐步拟合残差，提升拟合能力
- 广泛用于回归与分类
- 推荐使用XGBoost / LightGBM / CatBoost在生产中

---